In [1]:
# import packages
import csv
# import pandas
import numpy as np
import random

In [1]:
data_tr=pandas.read_csv(""/Users/fanerror/GitHub/DataFest2018/wdaycount.csv",header=0)
print (data_tr)

SyntaxError: EOL while scanning string literal (<ipython-input-1-05860aea0e96>, line 1)

In [3]:
# load training data
csvFile = open("/Users/fanerror/GitHub/DataFest2018/wdaycount.csv", "r")
reader = csv.reader(csvFile)
data_tr = list()
data_tr_lab = list()
for item in reader:
    # omit first line
    if reader.line_num == 1:
        continue
    data_tr.append(item[1])
    data_tr_lab.append(item[0])
csvFile.close()

# load test data
csvFile = open("./hw2data_1/reviews_te.csv", "r")
reader = csv.reader(csvFile)

data_te = list()
data_te_lab = list()
for item in reader:
    # omit first line
    if reader.line_num == 1:
        continue
    data_te.append(item[1])
    data_te_lab.append(item[0])
csvFile.close()

In [4]:
print(len(data_te),len(data_tr))

(320122, 1000000)


In [50]:
print(data_tr[0])

first time here food tastes great good environment choice between mimi s or here is a hard one yet there is wifi here families galore i didn t bring my son yet i see lots of families eating here for what you get you could go to sunset station yet the service was fast and prompt when i am in the area it s now a good place to fill in the answer to what do you want to eat


In [5]:
# Unigram representation
def Unigram(d):
    text = d.split(' ')
    wordcount = {}
    for item in text:
        if item not in wordcount.keys():
            wordcount[item] = 1
        else:
            wordcount[item] += 1
    return wordcount

# Term frequency-inverse document frequency (tf-idf) weighting
def tf_idf(d, D):
    
    ### define idf function
    def idf(t,D):
        count = 0
        for document in D:
            text = document.split(' ')
            if t in text:
                count += 1
        return len(D)/count
    
    wordcount = {}
    text = d.split(' ')
    for item in text:
        if item not in wordcount.keys():
            wordcount[item] = 1
        else:
            wordcount[item] += 1

    for key in wordcount.keys():
        idf_key = idf(key,D)
        wordcount[key] = wordcount[key]*np.log10(idf_key)
    return wordcount

# Bigram representation.
def Bigram(d):
    text = d.split(' ')
    wordcount = {}
    for iteration in range(len(text)-1):
        left = text[iteration]
        right = text[iteration+1]
        bigram = (left,right)
        if bigram not in wordcount.keys():
            wordcount[bigram] = 1
        else:
            wordcount[bigram] += 1
    return wordcount

In [6]:
# define operators
## because not every data point have same feature
def dict_plus(x,y,oper=1):
    '''
    x,y are both dictionary
    '''
    if oper == 0:
        oper = -1
    z = {}
    for key in set(x.keys()) | set(y.keys()):
        if key in set(x.keys()) & set(y.keys()):
            z[key] = x[key] + oper*y[key]
        elif key not in x.keys():
            z[key] = y[key]
        elif key not in y.keys():
            z[key] = x[key]
    return(z)

def dict_mult(x,y):
    '''
    x,y are both dictionary
    '''
    z = {}
    for key in set(x.keys()) & set(y.keys()):
        z[key] = x[key] * y[key]
    return(sum(z.values()))

In [10]:
def online(data,label):
    '''
    data: training data, list of dictionaries
        dictionary, key is feature and value is wordcount
    label: label corresponding to training data
    '''
    
    # Initialize w
    w = [{}]
    
    # create index
    index = range(len(data))
    # shuffle index
    random.shuffle(index)
    
    for i in index:
        sign_value = dict_mult(data[i],w[-1])
        if sign_value*label[i] <= 0:
            w.append(dict_plus(w[-1],data[i],label[i]))
    
    print("second shuffle")
    # second shuffle index
    random.shuffle(index)
    for i in index:
        sign_value = dict_mult(data[i],w[-1])
        if sign_value*label[i] <= 0:
            w.append(dict_plus(w[-1],data[i],label[i]))
        else:
            w.append(w[-1])
            
    return(w)

def online_perceptron(data,label,representations=1):
    '''
    data: raw training data, list
    label: raw label data, list
    representations:1 - Unigram representation
                    2 - Term frequency-inverse document frequency 
                    3 - Bigram representation
                    
    '''
    
    # convert raw data to list of dictionaries
    train_list= []
    train_label = []
    for i in range(len(data)):
        if representations == 1:
            train_list.append(Unigram(data[i]))
        elif representations == 3:
            train_list.append(Bigram(data[i]))
        elif representations == 2:
            train_list.append(tf_idf(data[i],data))
        if label[i]==1:
            train_label.append(1)
        else:
            train_label.append(-1)
    w = online(train_list,train_label)
    w_final = {}
    for wi in w:
        w_final = dict_plus(wi,w_final)
    for key in w_final.keys():
        w_final[key] = w_final[key] / (len(data) + 1)
    return (w_final)

In [48]:
def testmodel(data,label,w,representations=1):
    '''
    data: raw test data, list
    label: raw label data, list
    representations:1 - Unigram representation
                    2 - Term frequency-inverse document frequency 
                    3 - Bigram representation
                    
    '''
    
    # convert raw data to list of dictionaries
    test_list= []
    test_label = []
    for i in range(len(data)):
        if representations == 1:
            test_list.append(Unigram(data[i]))
        elif representations == 3:
            test_list.append(Bigram(data[i]))
        elif representations == 2:
            test_list.append(tf_idf(data[i],data))
        if label[i]==1:
            test_label.append(1)
        else:
            test_label.append(-1)
    count = 0        
    for i in range(len(test_list)):
        predict = dict_mult(w,test_list[i])
        if predict * test_label[i] < 0:
            count += 1
    print count
    print len(data)
    accuracy = format(float(count) / float(len(data)),'.5f')
    return (accuracy)

In [61]:
# main function
w1 = online_perceptron(data_tr[1:100],data_tr_lab[1:100],2)
accu1 = testmodel(data_te[1:100],data_te_lab[1:100],w1,2)
# w2 = online_perceptron(data_tr,data_tr_lab,2)
# accu2 = testmodel(data_te,data_te_lab,w2,2)
# w3 = online_perceptron(data_tr,data_tr_lab,3)
# accu3 = testmodel(data_te,data_te_lab,w3,3)
# print (accu1,accu2,accu3)

second shuffle
1
99


In [62]:
print accu1

0.01010
